First it'd be best to setup the environment. I've copied the raw courses from: 
https://gist.github.com/pat-coady/26fafa10b4d14234bfde0bb58277786d

Because this would take a while and be pretty tedious to do myself.

But we'd still need to convert this to some other format. 

I think the best thing to do first would be to setup the environment, then from there work on the reinforcement learning part of it.

Now that the environment is setup, we need to setup the target and behaviour policies.

The target will just be a greedy deterministic policy.

The behaviour policy will be epsilon greedy.

In [3]:
# Racetracks

# Left Race Track from Figure 5.5
big_course = ['WWWWWWWWWWWWWWWWWW',
              'WWWWooooooooooooo+',
              'WWWoooooooooooooo+',
              'WWWoooooooooooooo+',
              'WWooooooooooooooo+',
              'Woooooooooooooooo+',
              'Woooooooooooooooo+',
              'WooooooooooWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WoooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWooooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWoooooooWWWWWWWW',
              'WWWWooooooWWWWWWWW',
              'WWWWooooooWWWWWWWW',
              'WWWW------WWWWWWWW']

# Tiny course for debug

tiny_course = ['WWWWWW',
               'Woooo+',
               'Woooo+',
               'WooWWW',
               'WooWWW',
               'WooWWW',
               'WooWWW',
               'W--WWW',]

In [ ]:
import random

random.seed(1)

class RaceTrack:
    def __init__(self, course: list[str]):
        self.course = self._load_course(course)
        self.car_location = self._start()
        self.velocity = (0, 0)

    def _load_course(self, string_course: list[str]) -> np.array:
        course = np.zeros((len(string_course), len(string_course[0])), dtype=np.int16)
        course_dict = {"W": -1, "o": 0, "-": 1, "+": 2}

        for i in range(len(course)):
            for j in range(len(course[i])):
                course[i, j] = course_dict[string_course[i][j]]

        return course
    
    def _start(self) -> tuple[int, int]:
        rows, cols = np.where(self.course == 1)
        coords = list(zip(rows, cols))
        start = random.randint(0, len(coords)-1)

        return coords[start]
    
    def _restart(self):
        self.car_location = self._start()
        self.velocity = (0, 0)

    def get_course(self) -> np.array:
        return self.course
    
    def apply_velocity(self, action: tuple[int, int]):
        y_vel = min(max(self.velocity[0] + action[0], 0), 5) # Bound to between 0-5
        x_vel = min(max(self.velocity[1] + action[1], 0), 5)
        self.velocity = (y_vel, x_vel)

    def apply_movement(self) -> str:
        result = self.check_bounds()

        if result == "invalid":
            self._restart()
            return "restarted"
        elif result == "complete":
            return "done"
        else:
            return "continuing"
    
    def check_bounds(self) -> str:
        self.velocity = (5, 5)
        y_vel, x_vel = self.velocity
        y, x = self.car_location

        all_positions = []
        max_vel = max(x_vel, y_vel)
        y_vel_count, x_vel_count = y_vel, x_vel
        y_temp, x_temp = y, x

        for _ in range(max_vel):
            if y_vel_count > 0 and x_vel_count > 0:
                y_temp -= 1
                x_temp += 1
                all_positions.append((y_temp, x_temp))
                y_vel_count -= 1
                x_vel_count -= 1
            elif y_vel_count > 0:
                y_temp -= 1
                all_positions.append((y_temp, x_temp))
                y_vel_count -= 1
            else:
                x_temp += 1
                all_positions.append((y_temp, x_temp))
                x_vel_count -= 1

        all_position_values = []

        for i, j in all_positions:
            all_position_values.append(self.get_course()[i, j])

        if -1 in all_position_values:
            return "invalid"
        elif 2 in all_position_values:
            return "complete"
        else:
            self.car_location = all_positions[-1]
            return "valid"

    def random_episode(self) -> list:
        self.car_location = self._start()
        actions = [(x, y) for x in range(-1, 2) for y in range(-1, 2)]
        done = False
        path = []

        while not done:
            action = actions[random.randint(0, len(actions)-1)]
            self.apply_velocity(action)
            result = self.apply_movement()
            path.append(self.car_location)

            if result == "done":
                done = True
        
        return path



TypeError: zeros() missing required argument 'shape' (pos 0)

In [ ]:
import numpy as np


course = big_course
# Setup q values
q_values= np.zeros((len(course[0]), len(course), 5, 5, 3, 3)) # len(x), len(y), 5 speeds up and down, 3 possible actions for x and y axis. If it were state values, we'd just need v = x, y, 5, 5
c_values = np.zeros(q_values.shape) # Need the same shape for our running importance sampling sum
actions = [(x, y) for x in range(-1, 2) for y in range(-1, 2)]

target_policy = np.zeros((len(course[0]), len(course), 5, 5)) + 4 # Need an array for each state, default value set to (0, 0)
behaviour_policy = np.zeros(target_policy.shape) + (1/9) # Begin as equiprobable policy


